In [2]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks.
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import DBSCAN
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [3]:
IMG_SIZE = 100
LR = 1e-3
MODEL_NAME = 'fishCNN_oct_parallel-{}-{}.model'.format(LR, '2conv-basic')

In [4]:
# data creation
data=np.load("data_new_new.npy", allow_pickle=True)
print("Loaded data of shape {}.".format(data.shape))

Loaded data of shape (27370, 2).


In [5]:
#model creation
input_layer = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(input_layer, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
mp1 = max_pool_2d(convnet, 5)

fc1 = fully_connected(mp1, 1024, activation='relu')
do1 = dropout(fc1, 0.8)


fc2 = fully_connected(do1, 23, activation='softmax')
do2 = regression(fc2, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(do2, tensorboard_dir='log_mask')
print("Model built.")

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Model built.


In [6]:
#train data set up
train = data[:-int(len(data)/20)]
test = data[-int(len(data)/20):]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_y = [i[1] for i in test]
print("Train data formatting done. Initialising model fitting.")

Train data formatting done. Initialising model fitting.


In [7]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1220  | total loss: 0.17063 | time: 241.490s
| Adam | epoch: 003 | loss: 0.17063 - acc: 0.9547 -- iter: 25984/26002
Training Step: 1221  | total loss: 0.16696 | time: 242.098s
| Adam | epoch: 003 | loss: 0.16696 - acc: 0.9545 -- iter: 26002/26002
--


In [8]:
new_model = tflearn.DNN(mp1, session=model.session)
print("Second model created.")

Second model created.


In [9]:
partition1 = np.load("cnn_partitions/partition_1.npy", allow_pickle=True)

In [22]:
partitionList = [partition1[i:i + 500] for i in range(0, len(partition1), 500)]

In [23]:
len(partitionList)

20

In [24]:
preds = new_model.predict(partitionList[0])

In [28]:
idxs = list(range(20))

for j in idxs:
    npyname = "cnn_partitions/partition_" + str(j) + ".npy"
    partition = np.load(npyname, allow_pickle=True)
    partitionList = [partition[i:i + 500] for i in range(0, len(partition), 500)]
    print ("Created partition {}.".format(j))
    for i, item in enumerate(partitionList):
        preds = new_model.predict(item)
        fname = "cnn_outputs/res_" + str(j) + "_" + str(i) + ".npy"
        np.save(fname, preds)
        print(fname)

Created partition 0.
cnn_outputs/res_0_0.npy
cnn_outputs/res_0_1.npy
cnn_outputs/res_0_2.npy
cnn_outputs/res_0_3.npy
cnn_outputs/res_0_4.npy
cnn_outputs/res_0_5.npy
cnn_outputs/res_0_6.npy
cnn_outputs/res_0_7.npy
cnn_outputs/res_0_8.npy
cnn_outputs/res_0_9.npy
cnn_outputs/res_0_10.npy
cnn_outputs/res_0_11.npy
cnn_outputs/res_0_12.npy
cnn_outputs/res_0_13.npy
cnn_outputs/res_0_14.npy
cnn_outputs/res_0_15.npy
cnn_outputs/res_0_16.npy
cnn_outputs/res_0_17.npy
cnn_outputs/res_0_18.npy
cnn_outputs/res_0_19.npy
Created partition 1.
cnn_outputs/res_1_0.npy
cnn_outputs/res_1_1.npy
cnn_outputs/res_1_2.npy
cnn_outputs/res_1_3.npy
cnn_outputs/res_1_4.npy
cnn_outputs/res_1_5.npy
cnn_outputs/res_1_6.npy
cnn_outputs/res_1_7.npy
cnn_outputs/res_1_8.npy
cnn_outputs/res_1_9.npy
cnn_outputs/res_1_10.npy
cnn_outputs/res_1_11.npy
cnn_outputs/res_1_12.npy
cnn_outputs/res_1_13.npy
cnn_outputs/res_1_14.npy
cnn_outputs/res_1_15.npy
cnn_outputs/res_1_16.npy
cnn_outputs/res_1_17.npy
cnn_outputs/res_1_18.npy
cnn

cnn_outputs/res_15_16.npy
cnn_outputs/res_15_17.npy
cnn_outputs/res_15_18.npy
cnn_outputs/res_15_19.npy
Created partition 16.
cnn_outputs/res_16_0.npy
cnn_outputs/res_16_1.npy
cnn_outputs/res_16_2.npy
cnn_outputs/res_16_3.npy
cnn_outputs/res_16_4.npy
cnn_outputs/res_16_5.npy
cnn_outputs/res_16_6.npy
cnn_outputs/res_16_7.npy
cnn_outputs/res_16_8.npy
cnn_outputs/res_16_9.npy
cnn_outputs/res_16_10.npy
cnn_outputs/res_16_11.npy
cnn_outputs/res_16_12.npy
cnn_outputs/res_16_13.npy
cnn_outputs/res_16_14.npy
cnn_outputs/res_16_15.npy
cnn_outputs/res_16_16.npy
cnn_outputs/res_16_17.npy
cnn_outputs/res_16_18.npy
cnn_outputs/res_16_19.npy
Created partition 17.
cnn_outputs/res_17_0.npy
cnn_outputs/res_17_1.npy
cnn_outputs/res_17_2.npy
cnn_outputs/res_17_3.npy
cnn_outputs/res_17_4.npy
cnn_outputs/res_17_5.npy
cnn_outputs/res_17_6.npy
cnn_outputs/res_17_7.npy
cnn_outputs/res_17_8.npy
cnn_outputs/res_17_9.npy
cnn_outputs/res_17_10.npy
cnn_outputs/res_17_11.npy
cnn_outputs/res_17_12.npy
cnn_outputs/re